In [ ]:
import torch
import fastai
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from fastai.vision.all import *
from fastai.vision.augment import *

In [ ]:
#created dataset in colab

root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/data/MEMES/'
from fastai.vision import *
path = Path(base_dir)
path.mkdir(parents=True, exist_ok=True)

In [ ]:
#1000 memes

folder = 'meme'
jsonl_file_path = "facebook-hateful-meme-dataset/data/train.jsonl"
path = Path(base_dir)
destination_dir = path/folder

image_count = 0

with open(jsonl_file_path, 'r') as f:
    for line in f:
        entry = json.loads(line)

        image_url = entry["img"]
        print(image_url)

        source_file_path = Path(jsonl_file_path).parent / image_url
        print(source_file_path)
        filename = image_url.split("/")[-1]
        destination_file_path = destination_dir
        print(destination_file_path)
        shutil.copy(str(source_file_path), str(destination_file_path))

        image_count += 1

        if image_count == 1000:
            break


In [ ]:
import os
import shutil

#used a mini ImageNet dataset from Kaggle for not_memes

source_dir = 'imagenetmini-1000/imagenet-mini/train/'

destination_dir = 'gdrive/MyDrive/fastai-v3/data/MEMES/not_meme'

stop = 0
for folder_name in os.listdir(source_dir):
    folder_path = os.path.join(source_dir, folder_name)

    if os.path.isdir(folder_path):
        images = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

        if len(images) > 0:
            selected_image = images[0]

            source_file_path = os.path.join(folder_path, selected_image)
            destination_file_path = os.path.join(destination_dir, selected_image)

            shutil.copy(source_file_path, destination_file_path)

            print(f"Moved {selected_image} from {folder_name} to destination folder.")

    if stop == 1:
      break


In [ ]:
img_size = 460

#augment the images for better training

augmentations = [
    Brightness(max_lighting=0.3, p=0.5),
    Contrast(max_lighting=0.4, p=0.5),
    RandomErasing(p=0.3, sl=0.0, sh=0.2, min_aspect=0.3, max_count=1)]
    
data = ImageDataLoaders.from_folder(path='gdrive/MyDrive/fastai-v3/data/MEMES',
                               valid_pct = 0.2,
                               num_workers=0,
                               batch_tfms=augmentations,
                               item_tfms=Resize(img_size))
data.show_batch()

In [ ]:
learn = vision_learner(data,
                    resnet34,
                    metrics=[accuracy,error_rate])

learn.fit_one_cycle(4)

In [ ]:
#find the optimal learning rate
val = learn.lr_find()

learn.unfreeze()

In [ ]:
#lr_find() gives a plot, choose slice that has max loss slope

learn.fit_one_cycle(2, slice(3e-4,3e-3))